# In-Class Assignment

In this in-class assignment, you will be using a gene expression array dataset (gep.80samples.csv) from a cohort including patients with autism spectrum disorder (ASD) and their controls. You will start from establishing a set of differentially expressed genes between the patients and the controls using t-tests. From this gene set, you will be able to perform a Gene Ontology (GO) analysis to see if any GO terms are overrepresented in the gene set. Next, you will order all human genes from the most upregulated to the most downregulated genes in ASD patients compared to controls (you will use t-statistics for this purpose). Then, you can use gene set enrichment analysis (GSEA) to see whether any predefined set of genes (e.g., KEGG pathway genes) are enriched in upregulated or downregulated genes in ASD patients.

# Part 1. Differential gene expression analysis

In this section, you will use a gene expression array dataset [1] and compare gene expression levels between the patient and the control groups. Based on this comparison, you will list up differentially expressed genes with statistical significance.

[1] https://github.com/hms-dbmi/AISC2020/tree/master/Zak_lecture3

### 1a. Read the dataset

In [ ]:
# Read the array dataset file from above github URL. 
# Check.names = FALSE is helpful to avoid unnecessary, automatic changes in column names when they are numeric.
df <- read.csv(file = "copy_and_paste_the_file_location", header=T, as.is=T, sep=",", check.names=FALSE, dec=".")

In [ ]:
# Check how large is the dataset. You can use any functions including dim(), nrow(), ncol(), etc.

In [ ]:
# Below, you can see the first 10 columns and the first 10 rows. This way you can see an abstract structure of df.
df[c(1:10),c(1:10)]

### 1b. T-test comparisons for all genes between ASD and control groups.

In [ ]:
# Check how many patients and controls are in this dataset. You can use table() function.
# How many genes (probe IDs) do we have in this array dataset?

In [ ]:
# Let's compare the expression of "7896744" (in 6th column of this dataset) between ASD patients and controls.
# We will be performing t-test, using t.test() function. You see the detail of this function using below command.
?t.test

In [ ]:
# The most straightforward comparison could be done in this way:
t.test(df[,"7896744"][which(df$Dx == "ASD")], df[,"7896744"][which(df$Dx == "Control")])
# You can always substitute the column name ("7896744") into number: df[,6]

In [ ]:
# Another way to do it is as follows:
t.test(df[,6] ~ df$Dx)

In [ ]:
# We can retrieve specific values by using Dollar signs as follows:
t.test(df[,"7896744"][which(df$Dx == "ASD")], df[,"7896744"][which(df$Dx == "Control")])$p.value
t.test(df[,"7896744"][which(df$Dx == "ASD")], df[,"7896744"][which(df$Dx == "Control")])$statistic

### 1c. One way to do this is using "for loop".

In [ ]:
# We will iteratively perform t-test from column 6 to the last column, between the ASD patients and the controls.
# You can make a list of p-values from your for loop.

In [ ]:
# Name these p-values with their corresponding probe IDs (they are the column names of our df).

### 1d. Another way is using "apply()" function

In [ ]:
# Let's check the options of this function.
?apply

### 1e. Differentially expressed genes

In [ ]:
# How many genes have statistical significance (e.g., p-value < 0.05) in our list?

In [ ]:
# Make histograms to see the distribution of gene expression levels, 
# for any genes, i) with statistical significance, and ii) without it.

# Part 2. Gene Ontology Analysis (Using TopGO Package)

There are many R packages to perform Gene Ontology analysis. TopGO is one of them. Our aim here is to make a realistic sense of how the flow works. To achieve this, we will provide our codes to perform GO analysis (same for the next part: Part 3. GSEA analysis). By using "?" function and google, you can always find out what each step is doing, and how we could interpret the results. Make modifications of provided functions will also provide you insight. Details of TopGO package are described in below link.

https://www.bioconductor.org/packages/devel/bioc/vignettes/topGO/inst/doc/topGO.pdf

In [ ]:
sessionInfo()

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("topGo")
BiocManager::install("annotate")
BiocManager::install("hugene10sttranscriptcluster.db")
BiocManager::install("org.Hs.eg.db")

In [ ]:
library(topGO)
library(annotate)
library(hugene10sttranscriptcluster.db)
library(org.Hs.eg.db)

### 2a. Make a function to subset all genes to those with a p values < 0.05

In [ ]:
df <- read.csv(file = "https://github.com/hms-dbmi/AISC2020/raw/master/Zak_lecture3/gep.80samples.csv", header=T, as.is=T, sep=",", check.names=FALSE, dec=".")
list.pval = NULL
for (i in 6:ncol(df)){
    list.pval[i-5] = t.test(df[,i][df$Dx=="ASD"], df[,i][df$Dx=="Control"])$p.value
}
list.genes = list.pval
names(list.genes) = colnames(df)[6:ncol(df)]
head(list.genes)

In [ ]:
topGenes = function(x){
    return(x < 0.05)
}

sig.genes = list.genes[topGenes(list.genes)]
length(sig.genes)
head(sig.genes)

### 2b. Create a TopGO object

In [ ]:
sampleGOdata <- new("topGOdata", 
                    description = "Simple session", ontology = "BP",
                    allGenes = list.genes, geneSelectionFun = topGenes,
                    nodeSize = 2,
                    annot = annFUN.db, affyLib = "hugene10sttranscriptcluster.db")

In [ ]:
# inspect created object
class(sampleGOdata)
sampleGOdata

### 2c. Run GO analysis and filter the result

In [ ]:
# run gene ontology enrichment analysis
resultFisher <- runTest(sampleGOdata, statistic = "fisher")
# create result table with all enriched biological processes (BPs)
allRes <- GenTable(sampleGOdata, classicFisher = resultFisher, 
                   orderBy = "classicFisher", ranksOf = "classicFisher", 
                   topNodes = length(resultFisher@score))

In [ ]:
# subset to BPs with a p value < 0.05 (BP stands for "Biological Process")
result = allRes[which(allRes$classicFisher<0.05),]
str(result)
# print first BPs
head(result)
# print last BPs
tail(result)

# Part 3. Gene Set Enrichment Analysis (Using fgsea Package)

We will be using fgsea package to perform a GSEA analysis. Based on t-statistics, you can sort the genes from the one most highly expressed in the ASD patients compared to the controls, to the one that is expressed much lower in the patients compared to the control group. Then, you can use the ordered gene list to see which pathway genes are significantly enriched among those polarized genes. We will be using KEGG (Kyoto Encyclopedia of Genes and Genomes) pathway gene sets, to define the pathways for GSEA analysis. Details of FGSEA package is available in below link.

https://bioconductor.org/packages/release/bioc/vignettes/fgsea/inst/doc/fgsea-tutorial.html

In [ ]:
BiocManager::install("fgsea")
library(devtools)
library(fgsea)

In [ ]:
install.packages("aws.s3",
repos=c("cloudyr" = "http://cloudyr.github.io/drat"))
library("aws.s3")
# Set environmental variables
Sys.setenv("AWS_ACCESS_KEY_ID" = "XXXX",
"AWS_SECRET_ACCESS_KEY" = "XXXXX")
# Retrieve data
s3load("KEGGannot.RData", bucket ="dbmi-cumc")
s3load("GOannot.RData", bucket ="dbmi-cumc")

In [ ]:
# Load the KEGG pathway dataset and see what is in there.
ls()
keggpathways[1:10]

### 3a. Order genes based on t-statistics (ASD vs Control)

In [ ]:
# Remind how we called t-statistics in our differential gene expression analysis.
i = 6
t.test(df[,i][df$Dx=="ASD"], df[,i][df$Dx=="Control"])$statistic[[1]]

In [ ]:
list.tstat = NULL
for(i in 6:ncol(df)){
    list.tstat[i-5] <- t.test(df[,i][df$Dx=="ASD"], df[,i][df$Dx=="Control"])$statistic[[1]]
}
head(list.tstat)

In [ ]:
gsea.genes = list.tstat
names(gsea.genes) = colnames(df)[6:ncol(df)]
head(gsea.genes)
length(gsea.genes)

In [ ]:
ordered.genes <- sort(gsea.genes, decreasing = T)
head(ordered.genes)

### 3b. Creat fgsea object and run GSEA

In [ ]:
# using the sorted information, peform the gene set enrichment analysis
fgseaRes <- fgsea(pathways = keggpathways, stats = ordered.genes, minSize=15, maxSize=500, nperm=1000)

In [ ]:
# check the result based on the p-value
head(fgseaRes[order(pval),])
fgseaRes[fgseaRes$padj < 0.05,][order(pval),]
nrow(fgseaRes[fgseaRes$padj < 0.05,][order(pval),])

### 3c. GSEA plots for significantly enriched genes

In [ ]:
# plot enrichment of genes in the pathway of interest
par(mfrow = c(2,1))
plotEnrichment(keggpathways[["04010_MAPK signaling pathway"]], ordered.genes)
plotEnrichment(keggpathways[["04810_Regulation of actin cytoskeleton"]], ordered.genes)

In [ ]:
sessionInfo()